In [ ]:
import Visualization as viz
from ipywidgets import widgets
from IPython.display import display, clear_output
from bokeh.plotting import figure, show, output_notebook

output_notebook()

In [2]:
def on_button_click(b):
    global breaths

    clear_output()
    if results.alive:
        if breaths != 1:
            viz.update_database(breaths['_id'], w.selected_labels)

        breaths = results.next()
        df, breath_start, breath_end = viz.breath_viz(breaths['_id'])
        p=viz.make_plot(df, breath_start, breath_end)
        show(p)
    else:
        if breaths != 1:
            viz.update_database(breaths['_id'], w.selected_labels)
        clear_output()
        print('No more')

In [7]:
breaths = 1
results = viz.get_breaths(8)
w = widgets.SelectMultiple(description='Types', options=['Normal', 'DoubleStacked', 'PostDoubleStacked', 
                                                         'PressureFlowLimited', 'FlowAbnormal', 
                                                         'PrematureVentTermination', 'IneffectiveTrigger', 'Garbage'])
button = widgets.Button(description='Next')
container = widgets.HBox(children=[w, button])

button.on_click(on_button_click)
display(container)

No more
